In [13]:
# basic imports needed
import math
from collections import defaultdict
from collections import Counter
import pandas as pd

In [14]:
def create_index(documents):
    # Creating a dict to store the term freqs
    index = defaultdict(dict)

    # Creating a dict to store the document lengths
    doc_len = {}

    # Creating a set to store the unique terms
    terms = set()

    # Looping over the documents
    for i, document in enumerate(documents):

        # Looping over the terms in the document
        for term in document:

            # Change the term frequency
            index[term][i] = index[term].get(i, 0) + 1

            # Add the term to the set
            terms.add(term)

        # Store the length of the document
        doc_len[i] = len(document)

    # Compute the average document length
    avgdl = sum(doc_len.values()) / len(doc_len)

    # Compute the idf for each term
    idf = {}

    #print(terms)

    for term in terms:
        df = len(index[term])
        idf[term] = math.log(len(documents)/df)


    return index, doc_len, avgdl, idf


In [15]:
# MSMARCO SMALL dataset made by the code MSMARCO_SMALL.py

# Here we get the corpus, split the words and make a list of lists containing the passages.

collection = pd.DataFrame(pd.read_csv(r"C:\Users\Raul\Documents\Information-Retravel-System\MSMARCO_SMALL\collection_small.csv"))

print(collection['passage'][8770])

collection['passage'] = collection['passage'].str.split()

#print(collection)

list_of_lists_collection = collection['passage'].tolist()

list_of_lists_collection_pid = collection['pid'].tolist()

#print(list_of_lists)

tf = []

for doc in list_of_lists_collection:
  tf.append(dict(Counter(doc)))

index, doc_len, avgdl, idf = create_index(list_of_lists_collection)


home / medterms medical dictionary a-z list / intelligence quotient definition Medical Definition of Intelligence quotient Intelligence quotient: An attempt to measure the intelligence of someone. Abbreviated IQ. The IQ score is usually based upon the results of a written test.


In [16]:
# Print the results
# print(f"Index: {index}")
# print(f"Document Lengths: {doc_len}")
# print(f"Average Document Length: {avgdl}")
# print(f"Term Frequencies per doc in order: {tf}")
# print(f"Inverse Document Frequencies: {idf}")

# TF percentages rather than numbers, might be useful at some point. NOT USED NOW

In [17]:
def tf_perc(iindex):
    tf = []
    for doc in iindex.values():
        term_freq = {}
        total_words = sum(doc.values())
        for term in doc:
            term_freq[term] = doc[term] / total_words
        tf.append(term_freq)
    return tf

# Example usage
tf2 = tf_perc(index)
# print(tf2)

Code below is inspired by: https://medium.com/@evertongomede/understanding-the-bm25-ranking-algorithm-19f6d45c6ce

In [39]:
#BM25 ranking algorithm

# def bm25_ranking_index(index, query):
#     k1 = 1.2
#     b = 0.75
#     scores = defaultdict(int)
#     avgdl = sum(map(len, index.values())) / len(index)
#     doc = docs[doc_nr]
#     for term in query:
#         if term in index:
#             idf = len(index) / len(index[term])
#             for doc_nr, tf in index[term].items():
#                 score = idf * ((tf * (k1 + 1)) / (tf + k1 * (1 - b + b * (len(index[doc_nr]) / avgdl))))
#                 scores[doc_nr] += score
#     print(dict(sorted(scores.items(), key=lambda item: item[1], reverse=True)))
#     return dict(sorted(scores.items(), key=lambda item: item[1], reverse=True))

def bm25_ranking(tf, idf, docs, query):
    # Hyperparams to specify
    k1 = 1.2
    b = 0.75
    scores = {}

    # Looping through the different docs
    for doc_nr in range(len(docs)):
        score = 0
        doc = docs[doc_nr]
        
        # Loop for term in query in the doc
        for term in query:
            if str(term) in list(tf[doc_nr].keys()):

                # Calculating/updating the score
                score += idf[term] * ((tf[doc_nr][term] * (k1 + 1)) / (tf[doc_nr][term] + k1 * (1 - b + b * (len(doc) / avgdl))))

        scores[list_of_lists_collection_pid[doc_nr]] = score 
    
    # sort scores / ranking
    # print(scores)
    sorted_scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}
    return sorted_scores

# Use on MSMARCO

# using the queries and the corpus from earlier on, qrel file only used for eval later on.

queries = pd.DataFrame(pd.read_csv(r"C:\Users\Raul\Documents\Information-Retravel-System\MSMARCO_SMALL\queries_small.csv"))

#print(queries.head(5))

queries['query'] = queries['query'].str.split()

list_of_lists_query = queries['query'].tolist()

list_of_qids = queries['qid'].tolist()

#print(list_of_lists_collection[0])

In [40]:
qrel = pd.DataFrame(pd.read_csv(r"C:\Users\Raul\Documents\Information-Retravel-System\MSMARCO_SMALL\qrel_small.csv"))

scores = []
count = 0
eval_lists = []
for query in list_of_lists_query:
  #print(count)
  scores.append([list_of_qids[count], bm25_ranking(tf, idf, list_of_lists_collection, query)])
  
  count += 1
    

# print(scores[2])

{8756089: 17.03425836985787, 7833823: 16.901728523361186, 981344: 11.869049631462186, 6591016: 10.268837785824317, 2813709: 9.980594316345698, 4907148: 9.868766723437387, 2774118: 9.860781047556943, 2138671: 9.435650735486277, 6853091: 9.339921400223245, 5105518: 9.269381108969663, 8230583: 8.999200423088562, 5795853: 8.908291431958228, 36667: 8.72026997032548, 6093689: 8.674583217129674, 1724473: 8.458259977292387, 8159685: 8.43033399680712, 5208403: 8.425959074720986, 7477821: 8.38543571334718, 2991362: 8.340092535502093, 4020607: 8.198846800548047, 6969916: 8.134569906549814, 8230824: 8.126954852975446, 4665496: 7.938377198383034, 2877385: 7.899996565951297, 4190696: 7.850357562409174, 5959628: 7.775777288301659, 5802071: 7.6509428748129675, 5060619: 7.542666697571455, 2336324: 7.485184217258495, 3065204: 7.430847858019522, 3371735: 7.267848541336086, 3834395: 7.200809786842957, 5284283: 7.140001459046844, 7665753: 7.10226811647255, 2665200: 7.096235985144274, 7760881: 7.09249859978

{7614067: 14.22197546564806, 2318784: 10.306993203083959, 5697111: 9.599844227955803, 930379: 8.309905411119535, 2591151: 8.288694587158648, 4055164: 8.132748082217777, 3801887: 8.072158098026222, 5613270: 8.032153461901691, 899464: 7.894780973780091, 519930: 7.703468720025495, 3485497: 7.703166782451255, 8528559: 7.632576558306725, 1795996: 7.173953885170903, 7973922: 7.050065831054283, 3591169: 7.016645808335681, 652506: 6.9576195877359295, 3079969: 6.848456418566836, 7259172: 6.78646648537864, 4362085: 6.678810473126917, 7171385: 6.6268540732198735, 8446788: 6.623689153444224, 4702086: 6.465367127079005, 5612143: 6.363718517277428, 8620229: 6.2652398850291595, 2538015: 6.2036109383310425, 2247154: 6.117214213204481, 2184158: 6.106114783503784, 1410377: 5.974966990893081, 4498313: 5.934885061664446, 6855858: 5.92003582556467, 8442065: 5.92003582556467, 4755267: 5.916108865598771, 5558895: 5.875274752350123, 5121703: 5.7926560045082525, 5142528: 5.756159368836493, 2030426: 5.752662419

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{5175353: 12.848159268932855, 2970905: 10.50831753809427, 1301281: 10.084535945551332, 4998096: 9.084399061370942, 1485886: 6.412275724250738, 2637792: 6.347212213168613, 2511469: 6.237051994799514, 4915093: 6.169444892834955, 7007457: 6.1506985038382185, 1781917: 6.125673187751849, 4788532: 6.045971343779881, 6756765: 6.004172901230057, 2276226: 5.952713638380387, 14667: 5.935265613216354, 6321494: 5.874278441889872, 3944039: 5.828797320615496, 1189624: 5.786643408374196, 5214623: 5.715410317139001, 7205787: 5.664351930066966, 2299169: 5.66219849778882, 2106322: 5.578114699953714, 5275696: 5.566170631981086, 6076072: 5.4998685044945965, 3203473: 5.499701731604962, 3795152: 5.498360215949827, 2264507: 5.495786909376406, 8707475: 5.489491750997409, 3909841: 5.452771212320765, 579411: 5.421331996067287, 3141218: 5.402525343591712, 6897918: 5.3774348419842175, 3235281: 5.340472451932533, 8137361: 5.338522526175758, 6393618: 5.327760803267226, 2269572: 5.322865159409189, 6764729: 5.3178402

In [33]:
print(qrel.iloc[0, 3].dtype)

int64


In [34]:
df = pd.DataFrame({'col1': ['x', 'y', 'z', 'x'], 'col2': ['y', 'y', 'y', 'z']})

print(df.groupby('col1')['col2'].apply(lambda x: (x == 'y').all()).all())

False


In [35]:
from tqdm import tqdm
import numpy as np

eval_list = []

for query, docs in tqdm(scores):
    count = 0
    for doc, score in docs.items():
        count+=1
        if count > 20:
            break
        topic_doc_pair = (np.int64(query), np.int64(doc))

        if ((qrel['Topic'] == topic_doc_pair[0]) & (qrel['Document#'] == topic_doc_pair[1])).any():
        
            rel = 1
        
        else:
            
            rel = 0
            
        eval_list.append([query, rel, doc, list(docs.keys()).index(doc)+1, score, 'RUN1'])
        
print(eval_list[0])

100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [00:00<00:00, 91.78it/s]

[1049079, 0, 8756089, 1, 17.03425836985787, 'RUN1']


In [36]:
print(eval_list[1])

print(eval_list[2])

print(eval_list[3])

print(eval_list[4])

[1049079, 1, 7833823, 2, 16.901728523361186, 'RUN1']
[1049079, 0, 981344, 3, 11.869049631462186, 'RUN1']
[1049079, 0, 6591016, 4, 10.268837785824317, 'RUN1']
[1049079, 0, 2813709, 5, 9.980594316345698, 'RUN1']


In [12]:
def write_to_csv(file_path, data):
    with open(file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        for sublist in data:
            writer.writerow(sublist)

In [15]:
import csv

write_to_csv('BM25_evalcsv.csv', eval_list)
# with open('BM25_eval.txt', 'w') as f:
#     for sublist in eval_list:
#         line = ' '.join([str(elem) for elem in sublist])
#         f.write(line + '\n')

# END

In [ ]:
# from relevancy import relevancy_lookup
# import csv

# def process_qrel_file(qrel_path):
#     relevancies = relevancy_lookup()

#     with open(qrel_path) as file:
#         qrel_file = csv.reader(file, delimiter="\t")
#         for line in qrel_file:
#             query, document, relevancy = parse_qrel_line(line)
#             relevancies.add(query, document, relevancy)
#     return relevancies

# def parse_qrel_line(line):
#     #query_id, _, document_id, relevance
#     line = line[0].split()
#     return int(line[0]), line[2], int(line[3])

# qrel_path = "msmarco-docdev-qrels.tsv"
# relevancies = process_qrel_file(qrel_path)
# print(relevancies.relevancies)